## Visualization of Robot missions

In [1]:
from pathlib import Path
import pandas as pd
import contextily as cx
import folium
import branca.colormap as cm
import pickle

/tmp/ipykernel_577362/1877606597.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_folder_base = Path("../html/")
output_image_folder = output_folder_base / Path("./images/")
print(f"Writing website information to \n    {output_folder_base}")

Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results
Writing website information to 
    ../html


In [3]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results.head(3)


,lat_est,lon_est,yaw_est,hdop,nsats,lat_meas,lon_meas,depth,experiment_location,meas_running,...,CO2(ppm),airtemp in(degreeC),humidity in(rH),pressure in(mbar),airtemp out(degreeC),humidity out(rH),pressure out(mbar),PAR(umol m-2s-1),H2O temp(degreeC),position
datetime_AMT,,,,,,,,,,,,,,,,,,,,,
2023-03-04 20:00:00,-3.290738,-60.164893,111.314,2.59,6.0,-3.290751,-60.164883,7.617662,Iranduba,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:37,-3.290886,-60.165142,162.084,0.83,15.0,-3.290886,-60.165141,5.512197,Iranduba,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:38,-3.290886,-60.165140,172.000,0.78,17.0,-3.290886,-60.165140,5.475826,Iranduba,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cluster_position_file = input_data_folder / Path(cluster_positions_file_name)

with open(cluster_position_file, 'rb') as fp:
    centroids = pickle.load(fp)

centroids

{'Balbina': array([[ -1.90938492, -59.46105017],
        [ -1.90498015, -59.46979682],
        [ -1.90367786, -59.46055068]]),
 'Jandira': array([[ -3.23674311, -60.24114295],
        [ -3.23378505, -60.24179794],
        [ -3.2354515 , -60.2408596 ],
        [ -3.2338013 , -60.24178698]]),
 'Caldeirao': array([[ -3.24645146, -60.21718762],
        [ -3.25525976, -60.21942943],
        [ -3.25665423, -60.21853891],
        [ -3.25408879, -60.21902   ],
        [ -3.2460189 , -60.21627407],
        [ -3.25595593, -60.21851002]]),
 'Iranduba': array([[ -3.29182389, -60.1638196 ],
        [ -3.29371632, -60.16349051],
        [ -3.29160827, -60.16397365],
        [ -3.29146567, -60.16393856]])}

In [5]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [6]:
for location_name in df_results.experiment_location.unique():

    print(f"{location_name} ...")
    df_sep_pos = df_results[(df_results.experiment_location == location_name)]

    print(f"    ... providing key data for tables")
    df_key_param = df_sep_pos[df_sep_pos.meas_running].reset_index()\
            .groupby('corresponding_meas_file')\
                .agg(start=('datetime_AMT', 'min'),
                     end=('datetime_AMT', 'max'),
                     experiment_location = ('experiment_location', 'first'),
                     position = ('position', 'first'),
                     lat_est_mean=('lat_est', 'mean'),
                     lon_est_mean=('lon_est', 'mean'),
                     CO2_min=('CO2(ppm)', 'min'),
                     CO2_max=('CO2(ppm)', 'max'),
                     humidity_in_mean=('humidity in(rH)', 'mean'),
                     humidity_out_mean=('humidity out(rH)', 'mean'),
                     airtemp_in_mean=('airtemp in(degreeC)', 'mean'),
                     PAR_in_mean=('PAR(umol m-2s-1)', 'mean'),
                     airtemp_out_mean=('airtemp out(degreeC)', 'mean'))\
            .sort_values(by=['position', 'start'])
    
    print(f"    ... generating map background")
    map = folium.Map(location = [df_sep_pos.lat_est.median(),  
                                 df_sep_pos.lon_est.median()], 
                                 attr="Stamen Terrain",
                                 tiles='Stamen Terrain', zoom_start = 16, max_zoom=23)

    basemaps['Google Satellite Hybrid'].add_to(map)

    for index, row in df_sep_pos.iterrows():
        map.add_child(folium.CircleMarker(location = [row['lat_est'],row['lon_est']],
                                        color = 'gray',
                                        radius = 1, fill=True))
        
    for index, row in df_sep_pos[df_sep_pos.meas_running].iterrows():
        map.add_child(folium.CircleMarker(location = [row['lat_est'],row['lon_est']],
                                        color = 'red',
                                        radius = 1, fill=True))

    table_params = ['start', 'CO2_min', 'CO2_max', 'airtemp_in_mean', 'PAR_in_mean', 'position']

    print(f"    ... providing interactive elements")
    for index, location_cords in enumerate(centroids[location_name]):

        html_data = df_key_param[df_key_param.position== index][table_params]\
                        .reset_index()\
                        .sort_values(by=['position', 'start'])\
                        .to_html(classes='table table-stripped', 
                                index=False, float_format="%.2f")
        html = folium.Html(
            f"""
            <!DOCTYPE html>
            <html>
            <h5 align="center" style="color:black"><strong><u>Position {index} {location_name}</u><strong>
            </h5> 
            {html_data}
            <img src="./images/{location_name}_Positions_{index}.png" alt="HTML5 Icon" style="width:500px;height:500px;"> 
            </html>
            """, 
            script=True)

        popup  = folium.Popup(html, max_width=1000, max_height=1000)
        folium.vector_layers.Marker(location=[location_cords[0], location_cords[1]], 
                                    tooltip=f'Position {index}', popup = popup).add_to(map)

    map

    filename = f"{location_name.lower()}.html"
    html_folder = Path("../html/")
    html_file = Path.cwd() / html_folder / Path(filename)  
    print(f"    ... saving to \n        {html_file}")
        
    map.save(html_file)


Iranduba ...
    ... providing key data for tables
    ... generating map background


    ... providing interactive elements
    ... saving to 
        /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../html/iranduba.html


Balbina ...
    ... providing key data for tables
    ... generating map background


    ... providing interactive elements
    ... saving to 
        /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../html/balbina.html


Jandira ...
    ... providing key data for tables
    ... generating map background


    ... providing interactive elements
    ... saving to 
        /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../html/jandira.html


Caldeirao ...
    ... providing key data for tables
    ... generating map background


    ... providing interactive elements
    ... saving to 
        /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../html/caldeirao.html
